In [1]:
import pandas as pd


# File to Load

# School Data Victoria - SearchResults.xlsx. (2022). 
# Retrieved from https://asl.acara.edu.au/School-Search?state=VIC
school_data_to_load = "Resources/VictoriaSchoolData.xlsx"

# VCE Results Data Victoria - 2021SeniorSecondaryCompletionAndAchievementInformation.xlsx. (2021.) 
# Retrieved from https://www.vcaa.vic.edu.au/administration/research-and-statistics/Pages/SeniorSecondaryCompletion.aspx
school_results_data_to_load = "Resources/2021SeniorSecondaryCompletionAndAchievementInformation.xlsx"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_excel(school_data_to_load, skiprows=[0])
school_results_df = pd.read_excel(school_results_data_to_load, skiprows=[0,1,2,3,4,5,6,7,8,9], index_col=None, na_values=['NaN'])

In [2]:
# Dropping unusable columns in School Data df
cols = [0,3,4,7,9,10]
school_data_df_after_drop = school_data.drop(school_data.columns[cols],axis=1)

In [3]:
# Dropping unusable columns in School Results df
cols = [1,2,4,6,7,8,9,11,12,13,14]
school_results_df_after_drop = school_results_df.drop(school_results_df.columns[cols],axis=1)

In [4]:
# Adding a new column for school name and suburb in lower case
school_data_df_after_drop['School Name Lower'] = school_data_df_after_drop['School Name'].str.lower()
school_data_df_after_drop['Suburb Lower'] = school_data_df_after_drop['Suburb'].str.lower()

school_results_df_after_drop['School Name Lower'] = school_results_df_after_drop['School'].str.lower()
school_results_df_after_drop['Suburb Lower'] = school_results_df_after_drop['Locality'].str.lower()

# Merging dataframe School Data and School results by the newly created lower case School Name Lower AND Suburb Lower
merged_school_df = pd.merge(school_data_df_after_drop, school_results_df_after_drop, how='inner', 
                            left_on=["School Name Lower", "Suburb Lower"], 
                            right_on=["School Name Lower", "Suburb Lower"])

In [5]:
# Renaming Column headers for better readability
merged_school_df.rename(columns={"Number of VET certificates with 2021 enrolments": "VET Certs With 2021 Enrolments",
                                "Percentage of VCE students applying for tertiary places": "% Applying for Tertiary Study",
                                "Median VCE study score": "Median VCE Score",
                                "Percentage of study scores of 40 and over": "VCE 40+ %"}, inplace = True)

# Dropping duplicate columns
cols = [7,8,9,10]
merged_school_df = merged_school_df.drop(merged_school_df.columns[cols],axis=1)

In [6]:
# Bug fix. Identify values that are not int.
merged_school_df["Median VCE Score"].unique()

array(['30', '-', '28', '26', '24', '35', '29', '31', '33', '25', '32',
       '22', '37', '27', '34', '23', '21', '36', 'I/D', '19'],
      dtype=object)

In [7]:
# Bug fix. Replace values in df that are'-' and 'I/D' with a zero
merged_school_df = merged_school_df.replace(['-','I/D'],'0')

In [8]:
# Export this df to a csv so we can analyse the data in another notebook
merged_school_df.to_csv("Resources/Cleaned_VCE_results_by_school.csv", index=False)